In [1]:
# Enter the internet
import pandas as pd, time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Chrome ~> chromedriver
# Firefox ~> geckodriver
browser = webdriver.Chrome()

# Get Dollar quotation
browser.get('https://www.google.com/')
browser.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dolar', Keys.ENTER)
dollar_quotation = browser.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Get Euro quotation
browser.get('https://www.google.com/')
browser.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro', Keys.ENTER)
euro_quotation = browser.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Get Gold quotation
browser.get('https://www.melhorcambio.com/ouro-hoje')
gold_quotation = browser.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
gold_quotation = gold_quotation.replace(",", ".")

print('U$D = ', dollar_quotation, '£U = ', euro_quotation, 'GOLD = ', gold_quotation)

browser.quit()

U$D =  5.2539 £U =  6.165912218000001 GOLD =  296.46


In [2]:
# Import info to our database
table = pd.read_excel('data/Produtos.xlsx')
display(table)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [3]:
# Update Quotations
table.loc[table['Moeda'] == 'Dólar', 'Cotação'] = float(dollar_quotation)
table.loc[table['Moeda'] == 'Euro', 'Cotação'] = float(euro_quotation)
table.loc[table['Moeda'] == 'Ouro', 'Cotação'] = float(gold_quotation)

# Update Buy Price ~> Original Price x Quotation
table['Preço Base Reais'] = table['Preço Base Original'] * table['Cotação']

# Update Sell Price ~> Base Real Price x Margin
table['Preço Final'] = table['Preço Base Reais'] * table['Margem']
table['Preço Final'] = table['Preço Final'].map('{:.2f}'.format)

display(table)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.253900,5253.847461,1.40,7355.39
1,Carro Renault,4500.00,Euro,6.165912,27746.604981,2.00,55493.21
2,Notebook Dell,899.99,Dólar,5.253900,4728.457461,1.70,8038.38
3,IPhone,799.00,Dólar,5.253900,4197.866100,1.70,7136.37
4,Carro Fiat,3000.00,Euro,6.165912,18497.736654,1.90,35145.70
5,Celular Xiaomi,480.48,Dólar,5.253900,2524.393872,2.00,5048.79
6,Joia 20g,20.00,Ouro,296.460000,5929.200000,1.15,6818.58


# Update the new base price to the file

In [4]:
table.to_excel('data/Produtos_Novo.xlsx', index=False)